In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
def load_df(csv_path='./train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)
    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))
    return df
df = load_df()
df.drop(columns='adwordsClickInfo',axis=1,inplace=True)
keep_columns = df.columns[df.nunique()>1]
df = df.loc[:,keep_columns].copy()
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    return df
df = add_time_features(df)
df['transactionRevenue'] = df['transactionRevenue'].convert_objects(convert_numeric=True)
df['pageviews'] = df['pageviews'].convert_objects(convert_numeric=True)
df['transactionRevenue'].fillna(0,inplace=True)
df.drop(columns=['visitStartTime'],inplace=True)
df.drop(columns=['sessionId','visitId'],inplace=True)
df.drop(columns=['isMobile','deviceCategory','networkDomain','pageviews','medium','campaign','referralPath','keyword'
                   ,'adContent'],inplace=True)
df.drop(columns=['city','continent','metro','region','subContinent'],inplace=True)
df.sort_values(by=['transactionRevenue'], ascending=False,inplace=True)
df.drop_duplicates(subset='fullVisitorId',inplace=True)

AttributeError: 'DataFrame' object has no attribute 'time'

In [2]:
df.to_csv('./df.csv',index=False)